In [1]:
import csv
import os
import glob
import pandas as pd
import nltk
import string
import re
from nltk import ngrams
import multiprocessing
import time

In [2]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [3]:
def find_ngrams(sentence, n_list):
    """Magic n-gram function."""
    inp, grams = sentence.split(), []
    for n in n_list:
      grams += [' '.join(x) for x in zip(*[inp[i:] for i in range(n)])]
    return grams

In [4]:
# Change to Dir of file
#os.chdir("path")
os.chdir('C://Users//Ruben//Documents//Scriptie//Data//tng-environment')

list_csv = glob.glob("*.csv")
list_csv

['algh-1828-1914.csv',
 'algh_bigram_182801.csv',
 'algh_bigram_182802.csv',
 'algh_bigram_182803.csv',
 'algh_bigram_182804.csv',
 'algh_bigram_182805.csv',
 'algh_bigram_182806.csv',
 'algh_bigram_182807.csv',
 'algh_bigram_182808.csv',
 'algh_bigram_182809.csv',
 'algh_bigram_182810.csv',
 'algh_bigram_182811.csv',
 'algh_bigram_182812.csv',
 'algh_bigram_182901.csv',
 'algh_unigram_182801.csv',
 'algh_unigram_182802.csv',
 'algh_unigram_182803.csv',
 'algh_unigram_182804.csv',
 'algh_unigram_182805.csv',
 'algh_unigram_182806.csv',
 'algh_unigram_182807.csv',
 'algh_unigram_182808.csv',
 'algh_unigram_182809.csv',
 'algh_unigram_182810.csv',
 'algh_unigram_182811.csv',
 'algh_unigram_182812.csv',
 'algh_unigram_182901.csv',
 'arnc-1815-1914.csv']

In [6]:
# Function for extracting ngrams and generating tokenized files

def PreProcess(filename):
    # Iterate over CSVs
    df = pd.read_csv(filename, sep = "\t")
    print(filename + " imported")
    
    # Drop duplicates
    lbf = len(df)
    df = df.drop_duplicates(subset=['id'], keep='first')
    laf = len(df)
    print(str(lbf-laf) + " duplicates dropped")
    
    # Check av. number of articles per year
    start_year = int(df.date[0][0:4])
    end_year = int(df.date[len(df)-1][0:4])
    
    aay = len(df) / (end_year - start_year) 
    if aay < 5000:
        df.date = df.date.str.slice(0, 4)
        print('tokenizing years')

    if aay > 4999:
        df.date = df.date.str.replace("/", "")
        df.date = df.date.str.slice(0, 6)
        print('tokenizing months')

    df = df[df.date == 182902]
        
    # Iterate over years
    for year in sorted(list(set(df.date))):
        df_subset_year = df[df.date == year]
        df_subset_year = df_subset_year.reset_index(drop=True)
        print(str(year) + " = " + str(len(df_subset_year)) + " articles")
        bigram_df = pd.DataFrame()
        unigram_df = pd.DataFrame()
        
        list_tok_articles_year = list()
        
        # Iterate over Articles in Subsetted CSV
        for i in range(0,(len(df_subset_year.ocr) - 1)):
            
            
            article = df_subset_year.ocr[i]
            
            tokens = clean_and_split_str(article)
            list_tok_articles_year.append(tokens)
            
            
            # Get ngrams from tokenized_article
            list_ngrams = find_ngrams(tokens, [1,2])
            list_unigrams = [word for word in list_ngrams if len(word.split(" ")) == 1]
            list_bigrams = [word for word in list_ngrams if len(word.split(" ")) == 2]

        
            df_unigrams_article = pd.DataFrame(list_unigrams)
            df_unigrams_article['year'] = year
            df_unigrams_article['count'] = 1
            df_unigrams_article.columns = ['ngram', 'year', 'count']
            unigram_df = unigram_df.append(df_unigrams_article)
            
            
            df_bigrams_article = pd.DataFrame(list_bigrams)
            df_bigrams_article['year'] = year
            df_bigrams_article['count'] = 1
            df_bigrams_article.columns = ['ngram', 'year', 'count']
            bigram_df = bigram_df.append(df_bigrams_article)
            print(str(i) + "/" + str(len(df_subset_year)) + ": " + df_subset_year.date[i] + " " + df_subset_year.id[i] + " processed")
        
        #Write list of senttok articles to one file
        txt_name = filename[0:4] + "_lines_" + str(year) + ".txt"
                        
        with open(txt_name, 'w') as f:
            for item in list_tok_articles_year:
                f.write("%s\n" % item)

        
        ## Group Ngram DFS
        unigram_df = unigram_df.groupby(['ngram', 'year']).sum()
        unigram_df = unigram_df.reset_index()
        unigram_df = unigram_df[unigram_df['count'] > 1]
        fn = filename[0:4] + "_unigram_" + str(year) + ".csv"
        unigram_df.to_csv(fn, index = False)
    
        bigram_df = bigram_df.groupby(['ngram', 'year']).sum()
        bigram_df = bigram_df.reset_index()
        bigram_df = bigram_df[bigram_df['count'] > 1]
        fn = filename[0:4] + "_" + "_bigram_" + str(year) + ".csv"
        bigram_df.to_csv(fn, index = False)

In [41]:
bigram_df = pd.DataFrame()
unigram_df = pd.DataFrame()

year = '182902'
list_tok_articles_year = list()

for i in range(0,(len(df.ocr) - 1)):
    article = df.ocr[i]
    tokens = clean_and_split_str(article)
    list_tok_articles_year.append(tokens)
            
            
    # Get ngrams from tokenized_article
    list_ngrams = find_ngrams(tokens, [1,2])
    list_unigrams = [word for word in list_ngrams if len(word.split(" ")) == 1]
    list_bigrams = [word for word in list_ngrams if len(word.split(" ")) == 2]

        
    df_unigrams_article = pd.DataFrame(list_unigrams)
    if len(df_bigrams_article) > 0:
        df_unigrams_article['year'] = year
        df_unigrams_article['count'] = 1
        df_unigrams_article.columns = ['ngram', 'year', 'count']
        unigram_df = unigram_df.append(df_unigrams_article)
            
            
    df_bigrams_article = pd.DataFrame(list_bigrams)
    if len(df_bigrams_article) > 0:
        df_bigrams_article['year'] = year
        df_bigrams_article['count'] = 1
        df_bigrams_article.columns = ['ngram', 'year', 'count']
        bigram_df = bigram_df.append(df_bigrams_article)
    
    print(str(i) + "/" + str(len(df) - 1) + ": " + df.date[i] + " " + df.id[i] + " processed")
        

0/103: 182902 ddd:010067827:mpeg21:a0002 processed
1/103: 182902 ddd:010067827:mpeg21:a0011 processed
2/103: 182902 ddd:010067827:mpeg21:a0014 processed
3/103: 182902 ddd:010067827:mpeg21:a0007 processed
4/103: 182902 ddd:010067827:mpeg21:a0008 processed
5/103: 182902 ddd:010067827:mpeg21:a0009 processed
6/103: 182902 ddd:010067827:mpeg21:a0012 processed
7/103: 182902 ddd:010067827:mpeg21:a0013 processed
8/103: 182902 ddd:010067827:mpeg21:a0015 processed
9/103: 182902 ddd:010067827:mpeg21:a0016 processed
10/103: 182902 ddd:010067827:mpeg21:a0001 processed
11/103: 182902 ddd:010067827:mpeg21:a0003 processed
12/103: 182902 ddd:010067827:mpeg21:a0004 processed
13/103: 182902 ddd:010067827:mpeg21:a0010 processed
14/103: 182902 ddd:010067828:mpeg21:a0001 processed
15/103: 182902 ddd:010067828:mpeg21:a0004 processed
16/103: 182902 ddd:010067828:mpeg21:a0006 processed
17/103: 182902 ddd:010067828:mpeg21:a0007 processed
18/103: 182902 ddd:010067828:mpeg21:a0009 processed
19/103: 182902 ddd:010

In [44]:
bigram_df

,ngram,year,count
0,gemengde berigten,182902,1
1,berigten tegenwoordige,182902,1
2,tegenwoordige staat,182902,1
3,staat van,182902,1
4,van malacca,182902,1
5,malacca de,182902,1
6,de volgende,182902,1
7,volgende beschouwingen,182902,1
8,beschouwingen over,182902,1
9,over den,182902,1


In [50]:
df[103]

KeyError: 103